In [ ]:
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as pltfig

#Computes the multiplicative order of omega mod n. Omega needs to be relatively prime to n in order for this to terminate,
#which can be verified easily as part of a for-loop
def OrderMod(omega,n):
    i = omega
    j = 1

    while i != 1:
        i = i*omega % n
        j = j + 1

    return j
    
#Computes the multiplicative order of the matrix A mod n. det(A) needs to be relatively prime to n.
def computematrixorder(A,n):
    A = matrix(A)
    A = np.matrix(A)
    length = len(A)
    j = 1
    A = np.matrix(A)
    B = A
    while not (B == np.identity(length)).all():
        B = B.dot(A)
        B %= n
        j = j + 1
    return j

#Let K = \Q(\sqrt{-d}) be a quadratic imaginary field, where \O_K = \Z[\alpha] such that \alpha is in ther upper 
#half-plane and f is the minimal polynomial for \alpha. Let F be the companion matrix for f.
#Then for the modulus m, this computes the order of the matrix A = aI + bF in the ray class group of modulus m. However,
#this doesn't account for the case where A^e is in \O_K^\times for some power e, which needs to be checked and corrected.
def MatrixOrderinGalGp(d,m,a,b):
    D = -d % 4
    I = np.identity(2)
    if D == 0:
        print("d must not be divisible by 4")
        sys.exit()
    if D == 1:
        companion = -((d+1)//4)
        F = [[0, companion],[1,-1]]
        omega = complex(-0.5, np.sqrt(d)/2)
    else:
        F = [[0,-d], [1,0]]
        omega = complex(0, np.sqrt(d))
    A = np.add(np.multiply(I,a), np.multiply(F,b))
    A %= m
    det = int(round(np.linalg.det(A)))
    g = np.gcd(m, det)
    if g > 1:
        print("aI + bF is not invertible mod m")
        sys.exit()

    order = computematrixorder(A, m)
    A = A.astype(int)

    print("A = ", A)
    print("order(A) = "+str(order))
    
#Similar setup to the previous, except this computes A^power (this is useful when finding an element of a particular order)
def MatrixPowerinGalGp(d,m,a,b,power):
    D = -d % 4
    I = np.identity(2)
    if D == 0:
        print("d must not be divisible by 4")
        sys.exit()
    if D == 1:
        companion = -((d+1)//4)
        F = [[0, companion],[1,-1]]
        omega = complex(-0.5, np.sqrt(d)/2)
    else:
        F = [[0,-d], [1,0]]
        omega = complex(0, np.sqrt(d))
    A = np.add(np.multiply(I,a), np.multiply(F,b))
    A %= m
    det = int(round(np.linalg.det(A)))
    g = np.gcd(m, det)
    if g > 1:
        print("aI + bF is not invertible mod m")
        sys.exit()
    
    current_order = 1
    A = matrix(A.astype(int))
    B = A % m
    if power == 0:
        return matrix.identity(B.nrows())
    else:
        while current_order < power:
            B = B*A % m
            current_order += 1
        return B

#For a given order d, this computes the Laurent polynomial g_d showing up in the Duke--Garcia--Lutz Theorem
def DGLLaurentPoly(d):
    R.<x> = PolynomialRing(ZZ)
    Phi = cyclotomic_polynomial(d)
    Euler = Phi.degree()
    L = LaurentPolynomialRing(CC, Euler+1, 'z',order='deglex')
    L = L.remove_var('z0')
    z = list(L.gens())
    LaurPoly = 0
    
    for m in range(d):
        f = x^m % Phi
        coeffs = []
        for i in range(Euler):
            coeffs.append(f[i])
        prod = 1
        for j in range(Euler):
            prod = prod*(z[j]^coeffs[j])
        LaurPoly += prod
        
    return(LaurPoly)

#This does the same as the previous, except the output is ready to be compiled in LaTeX
def LatexDGLLaurentPoly(d):
    R.<x> = PolynomialRing(ZZ)
    Phi = cyclotomic_polynomial(d)
    Euler = Phi.degree()
    z = list(var('z_%d' % i) for i in range(1, Euler + 1))
    LaurPoly = 0
    
    for m in range(d):
        f = x^m % Phi
        coeffs = []
        for i in range(Euler):
            coeffs.append(f[i])
        prod = 1
        for j in range(Euler):
            prod = prod*(z[j]^coeffs[j])
        LaurPoly += prod
        
    return(latex(LaurPoly))